# Introducción a la manipulación de datos con pandas - II parte

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://numfocus.org/wp-content/uploads/2016/07/pandas-logo-300.png" width="400px" height="400px" />

> La clase pasada comenzamos a ver las funcionalidades de la librería [pandas](https://pandas.pydata.org/). Básicamente, aprendimos a cargar datos desde archivos `.csv`, aprendimos a seleccionar subcojuntos de estos datos mediante indización (obtener ciertas filas, o ciertas columnas, o ciertas filas y columnas determinadas). Finalmente, aprendimos a filtrar datos, es decir, a seleccionar registros de la base de datos que satisfagan cierta condición.

> Hoy nos enfocaremos a ver como reunir varias bases de datos que poseen información complementaria relativa a un mismo problema, para obtener una sola tabla con la información condensada.

Referencias:
- https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/
___

# 0. Motivación

En cualquier situación relacionada con ciencia de datos en la vida real, no pasarán más de 10 minutos sin que aparezca la necesidad de unir (`merge` o `join`) dos bases de datos complementarias para formar tu conjunto de datos para el análisis.

La unión de DataFrames es un proceso fundamental, que cualquier analista de datos en formación debe aprender. En esta clase aprenderemos cómo hacerlo, y en el camino responderemos las siguientes preguntas:

- ¿Qué son el `merge` o `join` de dos DataFrames?

- ¿Qué son `inner`, `outer`, `left` y `right` `joins`?



## Datos para la clase

En esta clase, seguiremos trabajando con los datos de la clase pasada:

En la carpeta "data" tenemos los archivos 

- "customers_data.csv": datos propios de los clientes de la empresa, 
- "sessions_data.csv": inicios de sesión de los clientes en la plataforma web de compras (similar a amazon),
- "transactions_data.csv": transacciones asociadas a cada inicio de sesión, y
- "products_data.csv": información de los productos. 

Podemos cargar cada uno de los archivos `.csv` como DataFrames de Pandas usando la función `read_csv()` de pandas, y examinar los contenidos de cada uno con el método `head()`.

In [13]:
# Importar pandas
import pandas as pd

In [15]:
# Cargar customers_data
customers = pd.read_csv("data/customers_data.csv", index_col=0)
customers.head()

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [19]:
customers.shape

(5, 4)

In [16]:
# Cargar sessions_data
sessions = pd.read_csv("data/sessions_data.csv", index_col=0)
sessions.head()

,session_id,customer_id,device,session_start
0,1,2,desktop,2014-01-01 00:00:00
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30


In [20]:
sessions.shape

(35, 4)

In [17]:
# Cargar transactions_data
transactions = pd.read_csv("data/transactions_data.csv", index_col=0)
transactions.head()

,transaction_id,session_id,transaction_time,product_id,amount
0,298,1,2014-01-01 00:00:00,5,127.64
1,2,1,2014-01-01 00:01:05,2,109.48
2,308,1,2014-01-01 00:02:10,3,95.06
3,116,1,2014-01-01 00:03:15,4,78.92
4,371,1,2014-01-01 00:04:20,3,31.54


In [41]:
transactions["transaction_id"].nunique()

500

In [21]:
transactions.shape

(500, 5)

In [18]:
# Cargar products_data
products = pd.read_csv("data/products_data.csv", index_col=0)
products.head()

,product_id,brand
0,1,B
1,2,B
2,3,B
3,4,B
4,5,A


In [22]:
products.shape

(5, 2)

## Bonus: formato de fechas y horas

Como vimos la vez pasada, es relevante en general un correcto manejo de las fechas y horas en las bases de datos.

Lo primero que se tiene que hacer para un correcto manejo de las fechas (y horas) es identificar las columnas o variables que contienen fechas. Por ejemplo, de la tabla `customers_data`:

In [25]:
transactions["transaction_time"]

0      2014-01-01 00:00:00
1      2014-01-01 00:01:05
2      2014-01-01 00:02:10
3      2014-01-01 00:03:15
4      2014-01-01 00:04:20
              ...         
495    2014-01-01 08:56:15
496    2014-01-01 08:57:20
497    2014-01-01 08:58:25
498    2014-01-01 08:59:30
499    2014-01-01 09:00:35
Name: transaction_time, Length: 500, dtype: object

Lo que sigue, es especificarle a pandas el formato en que vienen esas fechas con la función `to_datetime()` de pandas

In [27]:
# Ayuda en la función to_datetime
help(pd.to_datetime)

Help on function to_datetime in module pandas.core.tools.datetimes:

to_datetime(arg, errors='raise', dayfirst=False, yearfirst=False, utc=None, format=None, exact=True, unit=None, infer_datetime_format=False, origin='unix', cache=True)
    Convert argument to datetime.
    
    Parameters
    ----------
    arg : int, float, str, datetime, list, tuple, 1-d array, Series DataFrame/dict-like
        The object to convert to a datetime.
    errors : {'ignore', 'raise', 'coerce'}, default 'raise'
        - If 'raise', then invalid parsing will raise an exception.
        - If 'coerce', then invalid parsing will be set as NaT.
        - If 'ignore', then invalid parsing will return the input.
    dayfirst : bool, default False
        Specify a date parse order if `arg` is str or its list-likes.
        If True, parses dates with the day first, eg 10/11/12 is parsed as
        2012-11-10.
        with day first (this is a known bug, based on dateutil behavior).
    yearfirst : bool, defaul

In [28]:
transactions.dtypes

transaction_id        int64
session_id            int64
transaction_time     object
product_id            int64
amount              float64
dtype: object

In [29]:
# Especificar el formato de fechas en la tabla customers_data
transactions["transaction_time"] = pd.to_datetime(transactions["transaction_time"],
                                                  format="%Y-%m-%d %H:%M:%S",
                                                  errors="coerce")

In [30]:
transactions.dtypes

transaction_id               int64
session_id                   int64
transaction_time    datetime64[ns]
product_id                   int64
amount                     float64
dtype: object

Hagan esto mismo para todas las tablas:

In [33]:
sessions.dtypes

session_id        int64
customer_id       int64
device           object
session_start    object
dtype: object

In [34]:
sessions["session_start"]

0     2014-01-01 00:00:00
1     2014-01-01 00:17:20
2     2014-01-01 00:28:10
3     2014-01-01 00:44:25
4     2014-01-01 01:11:30
5     2014-01-01 01:23:25
6     2014-01-01 01:39:40
7     2014-01-01 01:55:55
8     2014-01-01 02:15:25
9     2014-01-01 02:31:40
10    2014-01-01 02:47:55
11    2014-01-01 03:04:10
12    2014-01-01 03:15:00
13    2014-01-01 03:28:00
14    2014-01-01 03:41:00
15    2014-01-01 03:49:40
16    2014-01-01 04:00:30
17    2014-01-01 04:14:35
18    2014-01-01 04:27:35
19    2014-01-01 04:46:00
20    2014-01-01 05:02:15
21    2014-01-01 05:21:45
22    2014-01-01 05:32:35
23    2014-01-01 05:44:30
24    2014-01-01 05:59:40
25    2014-01-01 06:17:00
26    2014-01-01 06:34:20
27    2014-01-01 06:50:35
28    2014-01-01 07:10:05
29    2014-01-01 07:27:25
30    2014-01-01 07:42:35
31    2014-01-01 08:02:05
32    2014-01-01 08:10:45
33    2014-01-01 08:24:50
34    2014-01-01 08:44:20
Name: session_start, dtype: object

In [36]:
# Especificar el formato de fechas en la tabla sessions
sessions["session_start"] = pd.to_datetime(sessions["session_start"],
                                           format="%Y-%m-%d %H:%M:%S",
                                           errors="coerce")

In [37]:
customers["join_date"]

0    2011-04-17 10:48:33
1    2012-04-15 23:31:04
2    2011-08-13 15:42:34
3    2011-04-08 20:08:14
4    2010-07-17 05:27:50
Name: join_date, dtype: object

In [38]:
customers["date_of_birth"]

0    1994-07-18
1    1986-08-18
2    2003-11-21
3    2006-08-15
4    1984-07-28
Name: date_of_birth, dtype: object

In [39]:
# Especificar el formato de fechas en la tabla customers
customers["join_date"] = pd.to_datetime(customers["join_date"],
                                        format="%Y-%m-%d %H:%M:%S",
                                        errors="coerce")
customers["date_of_birth"] = pd.to_datetime(customers["date_of_birth"],
                                            format="%Y-%m-%d",
                                            errors="coerce")

## Volviendo al problema ...

Hay columnas o variables que relacionan las tablas. Por ejemplo:

- Los clientes pueden iniciar sesión en la plataforma cuantas veces quieran. De manera que hay una relación uno a muchos entre las tablas "customers_data" y "sessions_data", mediante la variable "customer_id".

- Cuando se efectúa una transacción se supone que se está comprando un producto. Por lo tanto hay una relación uno a muchos entre las tablas "transactions_data"  y "products_data", mediante la variable "product_id".

## Problemas

Primero, quisieramos determinar cuál es el dispositivo preferido por zona.

Luego, quisiéramos determinar cuál es la marca preferida por zona.

Para resolver lo anterior necesitamos unir ("merge" o "join") nuestros datasets en uno solo para el análisis.

___
# 1. Uniendo dos DataFrames...

> “Merging” two datasets is the process of bringing two datasets together into one, and aligning the rows from each based on common attributes or columns.

Las palabras “merge” y “join” se usan indistintamente en Pandas, y en otros lenguajes como SQL y R. En Pandas, hay métodos separados “merge” y “join”, que realizan cosas similares (personalmente uso el método "merge"), y la función "merge".

Vamos a concentrarnos en el **primer problema**. En este escenario, necesitamos llevar a cabo los siguientes pasos:

- Para cada fila en el dataset `sessions_data`, debemos hacer una nueva columna que contenga el "zip_code" respectivo de cada cliente.
- Una vez hagamos esto, obtenemos la moda de los dispositivos para cada cliente.

**¿Podemos usar un ciclo?**

Claro que si. Se podría escribir un ciclo para esta tarea. Éste iría a través de cada fila en `sessions_data`, y a cada "user_id" asignar el valor de la nueva columna con la zona respectiva.

Sin embargo, usar ciclos haría nuestra tarea mucho más lenta y plagada de más código que el necesario que si se usara la función (método) `join()`.

De forma que, para estas situaciones, **nunca usar ciclos**.

Para ver cómo podemos hacer lo anterior, veamos la ayuda de la función `merge()` de pandas:

In [42]:
# Ayuda de merge()
help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(left, right, how: str = 'inner', on=None, left_on=None, right_on=None, left_index: bool = False, right_index: bool = False, sort: bool = False, suffixes=('_x', '_y'), copy: bool = True, indicator: bool = False, validate=None) -> 'DataFrame'
    Merge DataFrame or named Series objects with a database-style join.
    
    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    
    Parameters
    ----------
    left : DataFrame
    right : DataFrame or named Series
        Object to merge with.
    how : {'left', 'right', 'outer', 'inner'}, default 'inner'
        Type of merge to be performed.
    
        * left: use only keys from left frame, similar to a SQL left outer join;
          preserve key order.
        * right: use only keys from right fra

Ahora, veamos como podemos añadir el código postal a la tabla `sessions_data`, usando la función `merge()` de pandas:

In [43]:
# Uso de la función merge()
# sessions_with_zip = pd.merge(left=sessions,
#                              right=customers[['customer_id', 'zip_code']]
#                              on="customer_id")
sessions_with_zip = sessions.merge(right=customers[['customer_id', 'zip_code']],
                                   on="customer_id")

In [44]:
sessions_with_zip

,session_id,customer_id,device,session_start,zip_code
0,1,2,desktop,2014-01-01 00:00:00,13244
1,10,2,tablet,2014-01-01 02:31:40,13244
2,15,2,desktop,2014-01-01 03:41:00,13244
3,16,2,desktop,2014-01-01 03:49:40,13244
4,17,2,tablet,2014-01-01 04:00:30,13244
5,31,2,mobile,2014-01-01 07:42:35,13244
6,33,2,mobile,2014-01-01 08:10:45,13244
7,2,5,mobile,2014-01-01 00:17:20,60091
8,20,5,desktop,2014-01-01 04:46:00,60091
9,24,5,tablet,2014-01-01 05:44:30,60091


La función `merge()` es el objetivo principal de esta clase. Básicamente, la operación de unir dos DataFrames hace lo siguiente: 

- toma el DataFrame de la izquierda (argumento left=), 
- el DataFrame de la derecha (argumento right=),
- la columna donde se va a unir (argumento on=), y
- la forma en que se va a unir (argumento how=).

La variable en común sobre la que se hace la unión está especificada en el argumento `on`.

Con este resultado, podemos filtrar por zona y luego obtener la moda.

In [51]:
# Obtener la moda por zona
sessions_with_zip.loc[sessions_with_zip["zip_code"] == 13244, "device"].value_counts()

desktop    7
tablet     3
mobile     3
Name: device, dtype: int64

In [50]:
sessions_with_zip.loc[sessions_with_zip["zip_code"] == 60091, "device"].value_counts()

mobile     10
desktop     7
tablet      5
Name: device, dtype: int64

### Usando GroupBy

In [57]:
sessions_with_zip.groupby(by="zip_code")["device"].value_counts()

zip_code  device 
13244     desktop     7
          mobile      3
          tablet      3
60091     mobile     10
          desktop     7
          tablet      5
Name: device, dtype: int64

In [56]:
sessions_with_zip.groupby(by="zip_code")["device"].agg(lambda x: x.mode())

zip_code
13244    desktop
60091     mobile
Name: device, dtype: object

### ¿Qué son los tipos de unión inner, left, right y outer?

En nuestro ejemplo, unimos `sessions` con `customers` sobre la columna "custumers_id". En este caso, en ambas tablas existían todos los posibles valores de "customers_id" (1, 2, 3, 4, 5).

¿Qué pasa si esta situación no se cumple?

Por ejemplo, realicemos los siguientes cambios artificiales:

In [58]:
# artificial sessions_data
art_sessions = sessions[sessions['customer_id'] != 2]
# artificial customers_data
art_customers = customers[customers['customer_id'] != 5]

Por defecto, la operación `merge()` de pandas actpua con un merge tipo "inner". Un "inner merge", guarda únicamente los valores comúnes (en la columna especificada en el argumento `on=`) de ambos DataFrames.

Por ejemplo:

In [59]:
art_sessions

,session_id,customer_id,device,session_start
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30
5,6,1,tablet,2014-01-01 01:23:25
6,7,3,tablet,2014-01-01 01:39:40
7,8,4,tablet,2014-01-01 01:55:55
8,9,1,desktop,2014-01-01 02:15:25
10,11,4,mobile,2014-01-01 02:47:55
11,12,4,desktop,2014-01-01 03:04:10


In [60]:
art_customers

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15


In [61]:
# inner join
art_sessions.merge(right=art_customers, 
                   on="customer_id",
                   how="inner")

,session_id,customer_id,device,session_start,zip_code,join_date,date_of_birth
0,3,4,mobile,2014-01-01 00:28:10,60091,2011-04-08 20:08:14,2006-08-15
1,5,4,mobile,2014-01-01 01:11:30,60091,2011-04-08 20:08:14,2006-08-15
2,8,4,tablet,2014-01-01 01:55:55,60091,2011-04-08 20:08:14,2006-08-15
3,11,4,mobile,2014-01-01 02:47:55,60091,2011-04-08 20:08:14,2006-08-15
4,12,4,desktop,2014-01-01 03:04:10,60091,2011-04-08 20:08:14,2006-08-15
5,13,4,mobile,2014-01-01 03:15:00,60091,2011-04-08 20:08:14,2006-08-15
6,21,4,desktop,2014-01-01 05:02:15,60091,2011-04-08 20:08:14,2006-08-15
7,22,4,desktop,2014-01-01 05:21:45,60091,2011-04-08 20:08:14,2006-08-15
8,4,1,mobile,2014-01-01 00:44:25,60091,2011-04-17 10:48:33,1994-07-18
9,6,1,tablet,2014-01-01 01:23:25,60091,2011-04-17 10:48:33,1994-07-18


Los otros comportamientos se pueden revisar mejor a través de ejemplos:

In [62]:
# left join
art_sessions.merge(right=art_customers, 
                   on="customer_id",
                   how="left")

,session_id,customer_id,device,session_start,zip_code,join_date,date_of_birth
0,2,5,mobile,2014-01-01 00:17:20,NaN,NaT,NaT
1,3,4,mobile,2014-01-01 00:28:10,60091.0,2011-04-08 20:08:14,2006-08-15
2,4,1,mobile,2014-01-01 00:44:25,60091.0,2011-04-17 10:48:33,1994-07-18
3,5,4,mobile,2014-01-01 01:11:30,60091.0,2011-04-08 20:08:14,2006-08-15
4,6,1,tablet,2014-01-01 01:23:25,60091.0,2011-04-17 10:48:33,1994-07-18
5,7,3,tablet,2014-01-01 01:39:40,13244.0,2011-08-13 15:42:34,2003-11-21
6,8,4,tablet,2014-01-01 01:55:55,60091.0,2011-04-08 20:08:14,2006-08-15
7,9,1,desktop,2014-01-01 02:15:25,60091.0,2011-04-17 10:48:33,1994-07-18
8,11,4,mobile,2014-01-01 02:47:55,60091.0,2011-04-08 20:08:14,2006-08-15
9,12,4,desktop,2014-01-01 03:04:10,60091.0,2011-04-08 20:08:14,2006-08-15


In [63]:
# right join
art_sessions.merge(right=art_customers, 
                   on="customer_id",
                   how="right")

,session_id,customer_id,device,session_start,zip_code,join_date,date_of_birth
0,3.0,4,mobile,2014-01-01 00:28:10,60091,2011-04-08 20:08:14,2006-08-15
1,5.0,4,mobile,2014-01-01 01:11:30,60091,2011-04-08 20:08:14,2006-08-15
2,8.0,4,tablet,2014-01-01 01:55:55,60091,2011-04-08 20:08:14,2006-08-15
3,11.0,4,mobile,2014-01-01 02:47:55,60091,2011-04-08 20:08:14,2006-08-15
4,12.0,4,desktop,2014-01-01 03:04:10,60091,2011-04-08 20:08:14,2006-08-15
5,13.0,4,mobile,2014-01-01 03:15:00,60091,2011-04-08 20:08:14,2006-08-15
6,21.0,4,desktop,2014-01-01 05:02:15,60091,2011-04-08 20:08:14,2006-08-15
7,22.0,4,desktop,2014-01-01 05:21:45,60091,2011-04-08 20:08:14,2006-08-15
8,4.0,1,mobile,2014-01-01 00:44:25,60091,2011-04-17 10:48:33,1994-07-18
9,6.0,1,tablet,2014-01-01 01:23:25,60091,2011-04-17 10:48:33,1994-07-18


In [64]:
# outer join
art_sessions.merge(right=art_customers, 
                   on="customer_id",
                   how="outer")

,session_id,customer_id,device,session_start,zip_code,join_date,date_of_birth
0,2.0,5,mobile,2014-01-01 00:17:20,NaN,NaT,NaT
1,20.0,5,desktop,2014-01-01 04:46:00,NaN,NaT,NaT
2,24.0,5,tablet,2014-01-01 05:44:30,NaN,NaT,NaT
3,28.0,5,mobile,2014-01-01 06:50:35,NaN,NaT,NaT
4,30.0,5,desktop,2014-01-01 07:27:25,NaN,NaT,NaT
5,32.0,5,mobile,2014-01-01 08:02:05,NaN,NaT,NaT
6,3.0,4,mobile,2014-01-01 00:28:10,60091.0,2011-04-08 20:08:14,2006-08-15
7,5.0,4,mobile,2014-01-01 01:11:30,60091.0,2011-04-08 20:08:14,2006-08-15
8,8.0,4,tablet,2014-01-01 01:55:55,60091.0,2011-04-08 20:08:14,2006-08-15
9,11.0,4,mobile,2014-01-01 02:47:55,60091.0,2011-04-08 20:08:14,2006-08-15


Estos comportamientos son muy intuitivos a partir de sus valores.

Así mismo, sus usos son muy intuitivos (lo sabrán cuando se enfrenten a problemas reales).

### Resolvamos el problema 2

Determinar cuál es la marca preferida por zona.

In [69]:
sessions_with_zip = sessions.merge(right=customers[["customer_id", "zip_code"]],
                                   on="customer_id",
                                   how="inner")
sessions_with_zip.head()

,session_id,customer_id,device,session_start,zip_code
0,1,2,desktop,2014-01-01 00:00:00,13244
1,10,2,tablet,2014-01-01 02:31:40,13244
2,15,2,desktop,2014-01-01 03:41:00,13244
3,16,2,desktop,2014-01-01 03:49:40,13244
4,17,2,tablet,2014-01-01 04:00:30,13244


In [70]:
transactions_with_zip = transactions.merge(right=sessions_with_zip[["session_id", "zip_code"]],
                                           on="session_id",
                                           how="inner")
transactions_with_zip

,transaction_id,session_id,transaction_time,product_id,amount,zip_code
0,298,1,2014-01-01 00:00:00,5,127.64,13244
1,2,1,2014-01-01 00:01:05,2,109.48,13244
2,308,1,2014-01-01 00:02:10,3,95.06,13244
3,116,1,2014-01-01 00:03:15,4,78.92,13244
4,371,1,2014-01-01 00:04:20,3,31.54,13244
...,...,...,...,...,...,...
495,112,35,2014-01-01 08:56:15,5,55.42,13244
496,111,35,2014-01-01 08:57:20,3,34.87,13244
497,276,35,2014-01-01 08:58:25,1,10.94,13244
498,266,35,2014-01-01 08:59:30,5,19.86,13244


In [71]:
brand_with_zip = products.merge(right=transactions_with_zip[["product_id", "zip_code"]],
                                on="product_id",
                                how="inner")
brand_with_zip

,product_id,brand,zip_code
0,1,B,13244
1,1,B,60091
2,1,B,60091
3,1,B,60091
4,1,B,60091
...,...,...,...
495,5,A,13244
496,5,A,13244
497,5,A,13244
498,5,A,13244


In [74]:
brand_with_zip.loc[brand_with_zip["zip_code"] == 13244, "brand"].value_counts()

B    149
A     37
Name: brand, dtype: int64

In [75]:
brand_with_zip.loc[brand_with_zip["zip_code"] == 60091, "brand"].value_counts()

B    247
A     67
Name: brand, dtype: int64

Uso de los métodos `groupby()` y `agg()`:

In [76]:
brand_with_zip.groupby(by="zip_code")["brand"].value_counts()

zip_code  brand
13244     B        149
          A         37
60091     B        247
          A         67
Name: brand, dtype: int64

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>